## **Scraping de la web Books To Scrape**

In [ ]:
import requests #lo uso para hacer peticiones HTTP a la web
from bs4 import BeautifulSoup #permite parsear el HTML y extraer datos que necesitamos

url_base = "http://books.toscrape.com/"
r = requests.get(url_base) #hago la peticion, y guardo la respuesta
soup = BeautifulSoup(r.text, "html.parser") #convierte HTML a un objeto navegable

categorias = soup.select(".side_categories ul li ul li a") #permite usar selectores CSS

categorias_info = []
for cat in categorias:
    nombre = cat.text.strip() #evita espacios extra
    url = url_base + cat["href"] #arma la URL completa
    categorias_info.append({"nombre": nombre, "url": url})

categorias_info[:5]  #mostrar las primeras 5 para revisar


[{'nombre': 'Travel',
  'url': 'http://books.toscrape.com/catalogue/category/books/travel_2/index.html'},
 {'nombre': 'Mystery',
  'url': 'http://books.toscrape.com/catalogue/category/books/mystery_3/index.html'},
 {'nombre': 'Historical Fiction',
  'url': 'http://books.toscrape.com/catalogue/category/books/historical-fiction_4/index.html'},
 {'nombre': 'Sequential Art',
  'url': 'http://books.toscrape.com/catalogue/category/books/sequential-art_5/index.html'},
 {'nombre': 'Classics',
  'url': 'http://books.toscrape.com/catalogue/category/books/classics_6/index.html'}]

In [ ]:
def scrapear_libros_categoria(categoria_url): #recibe el URL de cada categoria
    libros = []
    page = 1 #para revisar varias paginas dentro de la categoria
    while True: #bucle para revisar paginas
        if page == 1:
            url = categoria_url
        else:
            url = categoria_url.replace("index.html", f"page-{page}.html")
        
        r = requests.get(url)
        if r.status_code != 200: #si no devuelve codigo 200, se rompe el bucle (INVESTIGAR)
            break
        
        soup = BeautifulSoup(r.text, "html.parser") #parseo
        libros_pagina = soup.select(".product_pod") #product_pod contiene cada libro en la pagina
        if not libros_pagina:
            break #si ya no hay libros en esta categoria...
        
        for libro in libros_pagina:
            titulo = libro.h3.a["title"]
            precio = libro.select_one(".price_color").text
            rating = libro.p["class"][1]  #"One", "Two", etc.
            link_relativo = libro.h3.a["href"]
            link = url_base + "catalogue/" + link_relativo.replace("../../", "")
            
            libros.append({
                "titulo": titulo,
                "precio": precio,
                "rating": rating,
                "url": link
            })
        page += 1
    return libros

In [ ]:
todos_los_libros = []

for cat in categorias_info:
    print("Scrapeando categoría:", cat["nombre"]) #permite ver que categoria esta scrapeando
    libros = scrapear_libros_categoria(cat["url"])
    for libro in libros:
        libro["categoria"] = cat["nombre"] #aseguramos que cada libro tenga la categoria correcta
    todos_los_libros.extend(libros) #usando extend agregamos cada elemento individualmente, append agregaria lista dentro de lista

print(f"Total de libros scrapados: {len(todos_los_libros)}") 

Scrapeando categoría: Travel
Scrapeando categoría: Mystery
Scrapeando categoría: Historical Fiction
Scrapeando categoría: Sequential Art
Scrapeando categoría: Classics
Scrapeando categoría: Philosophy
Scrapeando categoría: Romance
Scrapeando categoría: Womens Fiction
Scrapeando categoría: Fiction
Scrapeando categoría: Childrens
Scrapeando categoría: Religion
Scrapeando categoría: Nonfiction
Scrapeando categoría: Music
Scrapeando categoría: Default
Scrapeando categoría: Science Fiction
Scrapeando categoría: Sports and Games
Scrapeando categoría: Add a comment
Scrapeando categoría: Fantasy
Scrapeando categoría: New Adult
Scrapeando categoría: Young Adult
Scrapeando categoría: Science
Scrapeando categoría: Poetry
Scrapeando categoría: Paranormal
Scrapeando categoría: Art
Scrapeando categoría: Psychology
Scrapeando categoría: Autobiography
Scrapeando categoría: Parenting
Scrapeando categoría: Adult Fiction
Scrapeando categoría: Humor
Scrapeando categoría: Horror
Scrapeando categoría: Histo

In [4]:
for libro in todos_los_libros:
    precio_str = libro["precio"]
    # eliminar cualquier carácter que no sea número o punto
    precio_limpio = ''.join(c for c in precio_str if c.isdigit() or c == '.')
    libro["precio"] = float(precio_limpio)


In [ ]:
rating_map = {
    "One": 1,
    "Two": 2,
    "Three": 3,
    "Four": 4,
    "Five": 5
} #convertimos string a numero entero

for libro in todos_los_libros:
    libro["rating"] = rating_map.get(libro["rating"], 0)  #0 si no se reconoce

In [ ]:
import pandas as pd #libreria estandar de manipulacion de datos en python

df_libros = pd.DataFrame(todos_los_libros) #pd.Dataframe transforma la lista en tabla con columnas
df_libros.head(10) #mostramos las primeras 10 filas

,titulo,precio,rating,url,categoria
0,It's Only the Himalayas,45.17,2,http://books.toscrape.com/catalogue/../its-onl...,Travel
1,Full Moon over Noahâs Ark: An Odyssey to Mou...,49.43,4,http://books.toscrape.com/catalogue/../full-mo...,Travel
2,See America: A Celebration of Our National Par...,48.87,3,http://books.toscrape.com/catalogue/../see-ame...,Travel
3,Vagabonding: An Uncommon Guide to the Art of L...,36.94,2,http://books.toscrape.com/catalogue/../vagabon...,Travel
4,Under the Tuscan Sun,37.33,3,http://books.toscrape.com/catalogue/../under-t...,Travel
5,A Summer In Europe,44.34,2,http://books.toscrape.com/catalogue/../a-summe...,Travel
6,The Great Railway Bazaar,30.54,1,http://books.toscrape.com/catalogue/../the-gre...,Travel
7,A Year in Provence (Provence #1),56.88,4,http://books.toscrape.com/catalogue/../a-year-...,Travel
8,The Road to Little Dribbling: Adventures of an...,23.21,1,http://books.toscrape.com/catalogue/../the-roa...,Travel
9,Neither Here nor There: Travels in Europe,38.95,3,http://books.toscrape.com/catalogue/../neither...,Travel


## **Utilizamos la API Google Books para encontrar autores**

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

API_KEY = os.getenv("GOOGLE_BOOKS_API_KEY") #evita poner la clave directamente en el codigo

In [ ]:
import requests #solicitudes HTTP a la api
import time #para las pausas entre solicitudes

def buscar_autores_google(titulo):
    query = f"https://www.googleapis.com/books/v1/volumes?q={titulo}&key={API_KEY}"
    r = requests.get(query)
    if r.status_code == 200:
        data = r.json() #convierte el JSON en un diccionario python
        if "items" in data and len(data["items"]) > 0:
            # tomar el primer resultado
            autores = data["items"][0]["volumeInfo"].get("authors", ["Desconocido"])
            return autores
    return ["Desconocido"] #si no encuentra nada o hay error, devuelve desconocido

In [ ]:
autores_list = []

for i, libro in enumerate(todos_los_libros):
    titulo = libro["titulo"]
    autores = buscar_autores_google(titulo) #llama a la funcion definida
    autores_list.append(autores)
    
    # imprimir progreso
    print(f"{i+1}/{len(todos_los_libros)}: {titulo} → {autores}")
    
    # esperar 1 segundo entre requests
    time.sleep(1)

1/1000: It's Only the Himalayas → ['Desconocido']
2/1000: Full Moon over Noahâs Ark: An Odyssey to Mount Ararat and Beyond → ['Desconocido']
3/1000: See America: A Celebration of Our National Parks & Treasured Sites → ['Desconocido']
4/1000: Vagabonding: An Uncommon Guide to the Art of Long-Term World Travel → ['Desconocido']
5/1000: Under the Tuscan Sun → ['Desconocido']
6/1000: A Summer In Europe → ['Desconocido']
7/1000: The Great Railway Bazaar → ['Desconocido']
8/1000: A Year in Provence (Provence #1) → ['Peter Mayle']
9/1000: The Road to Little Dribbling: Adventures of an American in Britain (Notes From a Small Island #2) → ['Bill Bryson']
10/1000: Neither Here nor There: Travels in Europe → ['Desconocido']
11/1000: 1,000 Places to See Before You Die → ['Desconocido']
12/1000: Sharp Objects → ['Desconocido']
13/1000: In a Dark, Dark Wood → ['Desconocido']
14/1000: The Past Never Ends → ['Desconocido']
15/1000: A Murder in Time → ['Desconocido']
16/1000: The Murder of Roger Ackr

In [ ]:
import pandas as pd
#convierte la lista de todos los libros en tablas panda
df_libros = pd.DataFrame(todos_los_libros)
df_libros["autores"] = autores_list #agrego nueva columna

df_libros.head(10) #compruebo con los primeros 10

,titulo,precio,rating,url,categoria,autores
0,It's Only the Himalayas,45.17,2,http://books.toscrape.com/catalogue/../its-onl...,Travel,[Desconocido]
1,Full Moon over Noahâs Ark: An Odyssey to Mou...,49.43,4,http://books.toscrape.com/catalogue/../full-mo...,Travel,[Desconocido]
2,See America: A Celebration of Our National Par...,48.87,3,http://books.toscrape.com/catalogue/../see-ame...,Travel,[Desconocido]
3,Vagabonding: An Uncommon Guide to the Art of L...,36.94,2,http://books.toscrape.com/catalogue/../vagabon...,Travel,[Desconocido]
4,Under the Tuscan Sun,37.33,3,http://books.toscrape.com/catalogue/../under-t...,Travel,[Desconocido]
5,A Summer In Europe,44.34,2,http://books.toscrape.com/catalogue/../a-summe...,Travel,[Desconocido]
6,The Great Railway Bazaar,30.54,1,http://books.toscrape.com/catalogue/../the-gre...,Travel,[Desconocido]
7,A Year in Provence (Provence #1),56.88,4,http://books.toscrape.com/catalogue/../a-year-...,Travel,[Peter Mayle]
8,The Road to Little Dribbling: Adventures of an...,23.21,1,http://books.toscrape.com/catalogue/../the-roa...,Travel,[Bill Bryson]
9,Neither Here nor There: Travels in Europe,38.95,3,http://books.toscrape.com/catalogue/../neither...,Travel,[Desconocido]


## **Creamos la base de datos en donde almacenar todos los libros y autores**

In [2]:
import sqlite3

# Conexión a SQLite (crea el archivo si no existe)
conn = sqlite3.connect("books.db")
cursor = conn.cursor() #cursor = objeto que permite ejecutar comandos en SQL

# Tabla Libros
cursor.execute("""
CREATE TABLE IF NOT EXISTS Libros (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    titulo TEXT,
    precio REAL,
    rating INTEGER,
    url TEXT,
    categoria TEXT
)
""")

# Tabla Autores
cursor.execute("""
CREATE TABLE IF NOT EXISTS Autores (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    nombre TEXT UNIQUE
)
""")

# Tabla intermedia Libros_Autores
cursor.execute("""
CREATE TABLE IF NOT EXISTS Libros_Autores (
    libro_id INTEGER,
    autor_id INTEGER,
    FOREIGN KEY(libro_id) REFERENCES Libros(id),
    FOREIGN KEY(autor_id) REFERENCES Autores(id)
)
""") #asocia libro y autor, ambos deben existir

conn.commit()


In [ ]:
for _, row in df_libros.iterrows(): #cargamos los datos en la tabla libros uno por uno
    cursor.execute("""
    INSERT INTO Libros (titulo, precio, rating, url, categoria)
    VALUES (?, ?, ?, ?, ?)
    """, (row['titulo'], row['precio'], row['rating'], row['url'], row['categoria']))

conn.commit()

In [ ]:
for autores in df_libros['autores']: #de la lista de autores, se inserta a cada uno por separado
    for autor in autores:
        cursor.execute("""
        INSERT OR IGNORE INTO Autores (nombre) VALUES (?)
        """, (autor,))
        
conn.commit()

In [ ]:
for _, row in df_libros.iterrows(): #iterrows = acceder fila por fila al dataframe
    # obtener id del libro
    cursor.execute("SELECT id FROM Libros WHERE titulo = ?", (row['titulo'],)) #obtiene el id de cada libro
    libro_id = cursor.fetchone()[0]
    
    for autor in row['autores']: #recorremos cada autor del libro y buscamos su id
        cursor.execute("SELECT id FROM Autores WHERE nombre = ?", (autor,))
        autor_id = cursor.fetchone()[0]
        
        # insertar en tabla intermedia
        cursor.execute("""
        INSERT INTO Libros_Autores (libro_id, autor_id) VALUES (?, ?)
        """, (libro_id, autor_id))

conn.commit()

## **Diagrama UML de la Base de Datos**

El siguiente diagrama muestra las tablas y sus relaciones:

![](UMLdatabase.png)


## **Consultas varias**

#### **Consulta 1: Libros con puntuacion de 4 estrellas o mas**

In [3]:
query1 = """
SELECT titulo, precio
FROM Libros
WHERE rating >= 4;
"""
cursor.execute(query1)
cursor.fetchall()

[('Full Moon over Noahâ\x80\x99s Ark: An Odyssey to Mount Ararat and Beyond',
  49.43),
 ('A Year in Provence (Provence #1)', 56.88),
 ('1,000 Places to See Before You Die', 26.08),
 ('Sharp Objects', 47.82),
 ('The Past Never Ends', 56.5),
 ('The Murder of Roger Ackroyd (Hercule Poirot #4)', 44.1),
 ('A Time of Torment (Charlie Parker #14)', 48.35),
 ('Murder at the 42nd Street Library (Raymond Ambler #1)', 54.36),
 ('What Happened on Beale Street (Secrets of the South Mysteries #2)', 25.37),
 ("The Bachelor Girl's Guide to Murder (Herringford and Watts Mysteries #1)",
  52.3),
 ('Delivering the Truth (Quaker Midwife Mystery #1)', 20.89),
 ('The Mysterious Affair at Styles (Hercule Poirot #1)', 24.8),
 ('The Silkworm (Cormoran Strike #2)', 23.05),
 ("The No. 1 Ladies' Detective Agency (No. 1 Ladies' Detective Agency #1)",
  57.7),
 ('The Girl You Lost', 12.29),
 ('A Flight of Arrows (The Pathfinders #2)', 55.53),
 ('Mrs. Houdini', 30.25),
 ('The Marriage of Opposites', 28.08),
 ('A Pa

##### Comentario: Revela los libros con 4 estrellas o mas, los cuales definitivamente valen la pena leer

#### **Consulta 2: Libros que cuesten menos de 20 euros**

In [4]:
query2 = """
SELECT titulo, precio
FROM Libros
WHERE precio < 20
ORDER BY precio ASC;
"""
cursor.execute(query2)
cursor.fetchall()

[('An Abundance of Katherines', 10.0),
 ('The Origin of Species', 10.01),
 ('The Tipping Point: How Little Things Can Make a Big Difference', 10.02),
 ('Patience', 10.16),
 ('Greek Mythic History', 10.23),
 ('The Fellowship of the Ring (The Lord of the Rings #1)', 10.27),
 ('History of Beauty', 10.29),
 ('The Lucifer Effect: Understanding How Good People Turn Evil', 10.4),
 ('NaNo What Now? Finding your editing process, revising your NaNoWriMo book and building a writing career through publishing and beyond',
  10.41),
 ('Pet Sematary', 10.56),
 ('I Am Pilgrim (Pilgrim #1)', 10.6),
 ('Counting Thyme', 10.62),
 ('The Complete Maus (Maus #1-2)', 10.64),
 ('The Project', 10.65),
 ('Are We There Yet?', 10.66),
 ('Tastes Like Fear (DI Marnie Rome #3)', 10.69),
 ('Miss Peregrineâ\x80\x99s Home for Peculiar Children (Miss Peregrineâ\x80\x99s Peculiar Children #1)',
  10.76),
 ('Green Eggs and Ham (Beginner Books B-16)', 10.79),
 ('A Clash of Kings (A Song of Ice and Fire #2)', 10.79),
 ('Adul

##### Comentario: Gran variedad de libros para un presupuesto ajustado

#### **Consulta 3: Los 5 libros mas baratos**

In [5]:
query3 = """
SELECT titulo, categoria, precio
FROM Libros
ORDER BY precio ASC
LIMIT 5;
"""
cursor.execute(query3)
cursor.fetchall()

[('An Abundance of Katherines', 'Young Adult', 10.0),
 ('The Origin of Species', 'Science', 10.01),
 ('The Tipping Point: How Little Things Can Make a Big Difference',
  'Add a comment',
  10.02),
 ('Patience', 'Sequential Art', 10.16),
 ('Greek Mythic History', 'Default', 10.23)]

##### Comentario: top 5 definitivo para el ahorrador

#### **Consulta 4: Libros que cuestan mas de 20, pero tienen rating de 2 o menos estrellas**

In [8]:
query4 = """
SELECT titulo, precio, rating
FROM Libros
WHERE precio >20 and rating <=2
ORDER BY precio DESC
LIMIT 5;
"""
cursor.execute(query4)
cursor.fetchall()

[('Civilization and Its Discontents', 59.95, 2),
 ('Thomas Jefferson and the Tripoli Pirates: The Forgotten War That Changed American History',
  59.64,
  1),
 ('The Improbability of Love', 59.45, 1),
 ("Miller's Valley", 58.54, 2),
 ("The Lover's Dictionary", 58.09, 2)]

##### Comentario: Libros que ademas de ser caros, probablemente son malos (y queremos evitar)

#### **Consulta 5: Categorias con mas libros**

In [23]:
cursor.execute("""
SELECT categoria, COUNT(id) AS cantidad
FROM Libros
GROUP BY categoria
ORDER BY cantidad DESC
LIMIT 5;
""")
cursor.fetchall()


[('Default', 152),
 ('Nonfiction', 110),
 ('Sequential Art', 75),
 ('Add a comment', 67),
 ('Fiction', 65)]

##### Comentario: Donde los escritores publican más rápido de lo que podés leer.

#### **Consulta sin indice vs consulta con indice**

In [ ]:
query = """
SELECT titulo, precio, rating
FROM Libros
WHERE precio < 25 AND rating >= 3
ORDER BY precio ASC;
"""

cursor.execute(query)
cursor.fetchall()
#probamos consultar libros con precio menor a 25 y rating mayor o igual a 3 estrellas

[('An Abundance of Katherines', 10.0, 5),
 ('The Origin of Species', 10.01, 4),
 ('Patience', 10.16, 3),
 ('Greek Mythic History', 10.23, 5),
 ('History of Beauty', 10.29, 4),
 ('NaNo What Now? Finding your editing process, revising your NaNoWriMo book and building a writing career through publishing and beyond',
  10.41,
  4),
 ('Pet Sematary', 10.56, 3),
 ('I Am Pilgrim (Pilgrim #1)', 10.6, 4),
 ('The Complete Maus (Maus #1-2)', 10.64, 3),
 ('Are We There Yet?', 10.66, 3),
 ('Green Eggs and Ham (Beginner Books B-16)', 10.79, 4),
 ('A Clash of Kings (A Song of Ice and Fire #2)', 10.79, 3),
 ('The Power Greens Cookbook: 140 Delicious Superfood Recipes', 11.05, 5),
 ('Reservations for Two', 11.1, 3),
 ('Dear Mr. Knightley', 11.21, 5),
 ('City of Fallen Angels (The Mortal Instruments #4)', 11.23, 4),
 ('The Darkest Corners', 11.33, 5),
 ('Naturally Lean: 125 Nourishing Gluten-Free, Plant-Based Recipes--All Under 300 Calories',
  11.38,
  5),
 ('Brilliant Beacons: A History of the America

In [ ]:
import time

def medir_tiempo(query, repeticiones=5): #lo ejecutamos 5 veces, para obtener un promedio
    tiempos = []
    for _ in range(repeticiones):
        inicio = time.time() #registra el momento justo antes de ejecutar la consulta
        cursor.execute(query)
        cursor.fetchall()
        tiempos.append(time.time() - inicio)
    return sum(tiempos) / len(tiempos)

tiempo_sin_indice = medir_tiempo(query)
print("⏱️ Promedio SIN índice:", round(tiempo_sin_indice, 6), "segundos")

⏱️ Promedio SIN índice: 0.001138 segundos


In [ ]:
cursor.execute("CREATE INDEX IF NOT EXISTS idx_libros_precio ON Libros(precio);") #creamos indice
conn.commit()

In [ ]:
tiempo_con_indice = medir_tiempo(query) #volvemos a medir despues de los cambios
print("⚡ Promedio CON índice:", round(tiempo_con_indice, 6), "segundos")

⚡ Promedio CON índice: 0.000594 segundos


In [15]:
print(f"🔍 Diferencia: {round(tiempo_sin_indice - tiempo_con_indice, 6)} segundos")

🔍 Diferencia: 0.000544 segundos


**Análisis del efecto del índice**:
La consulta selecciona libros con precio menor a £25 y rating mayor o igual a 3.
Sin índice, la base recorre todas las filas para evaluar la condición (precio < 25).
Luego de crear el índice en la columna precio, SQLite puede acceder directamente a las filas relevantes,
reduciendo el tiempo de búsqueda.
En este dataset pequeño la diferencia es mínima, pero en bases reales con miles de registros la mejora
sería significativa, pasando de segundos a milisegundos.